In [ ]:
from ti_experiments.train_model_concat_ti import main

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from models import Transformer
from ti_experiments.configs.cfg_iw import config as default_config
from ti_experiments.train_model_concat_ti import eval_at_all_distances
from itertools import product
from sklearn.decomposition import PCA
from plotting_utils import TI_per_pair_plot
import adjustText

sns.set_context("paper", font_scale=3)

metrics = main(default_config, wandb_proj='iw_transinf', seed=42)

iters = np.arange(0, default_config.train.niters, default_config.log.logging_interval)



In [ ]:
fig, ax = plt.subplots()
plt.plot(iters, metrics['loss'])
plt.ylabel('Loss')
plt.xlabel('Training steps')
plt.tight_layout()
plt.savefig('loss.png')
plt.show()

fig, ax = plt.subplots()
plt.plot(iters, metrics['holdout_accuracy'])
plt.ylabel('Accuracy (adjacent queries)')
plt.xlabel('Training steps')
plt.tight_layout()
plt.savefig('accuracy.png')
plt.show()


In [ ]:



cfg = default_config
cfg.log.log_to_wandb = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

cfg.model.out_dim = 1
model_dir = '/Users/jessegeerts/Projects/icl-generalization/checkpoints/iw_transinf/uuh8wjk6/'
model_path = os.path.join(model_dir, 'model_5000.pt')
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found: {model_path}")
items = torch.load(os.path.join(model_dir, 'fixed_items.pt'))

model = Transformer(config=default_config.model)
model.load_state_dict(torch.load(model_path))
model.eval()

correct_matrix, holdout_batch, pred_matrix, ranks, model_activations = eval_at_all_distances(cfg,
                                                                                             device,
                                                                                             model,
                                                                                             5000,
                                                                                             get_hiddens=True,
                                                                                             leave_one_out=True,
                                                                                             items=items)

# Example ranks
item_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G']  # Replace with your actual ranks

# Generate the labels for off-diagonal elements
labels = []
symb_distance = []
for i, j in product(ranks, ranks):
    if i == j:
        continue  # Skip diagonal elements (AA, BB, etc.)
    label = item_labels[i] + item_labels[j]  # Concatenate to form labels like 'AB', 'AC', etc.
    labels.append(label)
    symb_distance.append(i-j)


len(model_activations)

final_token_activations = [model_activations[i]['hidden_activations'][-1].mean(axis=0)[-1].detach() for i in range(len(model_activations))]
final_token_activations = np.array([np.array(i) for i in final_token_activations])


pca = PCA(n_components=2)  # Reduce to 2 dimensions for visualization
X_pca = pca.fit_transform(final_token_activations)  # Shape: [P, 2]

plt.figure(figsize=(10, 8))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=symb_distance, cmap='coolwarm', s=100)

# Add labels for each point
for i, inp in enumerate(labels):
    plt.annotate(inp, (X_pca[i, 0], X_pca[i, 1]))

plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('PCA of final layer Query Token Representations')
plt.grid(True)
plt.show()


In [ ]:
explained_variance = pca.explained_variance_ratio_
print(f"Variance explained by PC1: {explained_variance[0]:.4f} ({explained_variance[0]*100:.2f}%)")
print(f"Variance explained by PC2: {explained_variance[1]:.4f} ({explained_variance[1]*100:.2f}%)")
print(f"Total variance explained: {sum(explained_variance):.4f} ({sum(explained_variance)*100:.2f}%)")



In [ ]:
TI_per_pair_plot(pred_matrix)

In [ ]:
sns.set_context("paper", font_scale=3, rc={"lines.linewidth": 3})

fig, axs = plt.subplots(2, 2, figsize=(15, 10))
axs = axs.flatten()

axs[0].plot(iters, metrics['loss'])
axs[0].set_ylabel('Loss')
axs[0].set_xlabel('Training steps')

TI_per_pair_plot(pred_matrix, ax=axs[1])
axs[1].tick_params(axis='x', labelsize=14)  # x-axis tick labels
axs[1].set_title('Predictions by Symbolic Distance')

axs[2].plot(iters, metrics['holdout_accuracy'])
axs[2].set_ylabel('Accuracy (training)')
axs[2].set_xlabel('Training steps')

plt.sca(axs[3])
axs[3].scatter(X_pca[:, 0], X_pca[:, 1], c=symb_distance, cmap='coolwarm', s=100)

# Add labels for each point
txts = []
for i, inp in enumerate(labels):
    txt = plt.annotate(inp, (X_pca[i, 0], X_pca[i, 1]), fontsize=15)
    txts.append(txt)

plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('PCA of L2 Query Representations')

adjustText.adjust_text(txts)
plt.tight_layout()

plt.savefig('iw_transinf.pdf')

In [ ]:
# train N times